In [1]:
f1=[[147, 123], [556, 207]]
f2=[[146, 122], [556, 209], [653, 174]]


In [2]:
from sklearn.metrics.pairwise import euclidean_distances

In [3]:
x= euclidean_distances(f1,f2)

In [4]:
x

array([[  1.41421356, 417.94377612, 508.56366366],
       [418.71828238,   2.        , 102.45974819]])

In [5]:
import numpy as np

In [6]:
min_val,max_val = x.min(),x.max()

In [8]:
normalized = (x-min_val)/(max_val-min_val)

In [17]:
for i in range(len(normalized)):
    print(list(normalized[i]))
    print(":@@@@@@@@@@@@@@@@@@@")

[0.0, 0.8213152207434937, 1.0]
:@@@@@@@@@@@@@@@@@@@
[0.8228423963278771, 0.0011550568328700386, 0.19924212598743657]
:@@@@@@@@@@@@@@@@@@@


In [18]:
for i in range(len(normalized)):
    print(np.argmin(normalized[i]))
    print(":@@@@@@@@@@@@@@@@@@@")

0
:@@@@@@@@@@@@@@@@@@@
1
:@@@@@@@@@@@@@@@@@@@


In [83]:
def face_proximity_detecton(frame1,frame2,proximity_threshold):
    distanct_matrix = euclidean_distances(frame1,frame2)
    min_val,max_val = distanct_matrix.min(),distanct_matrix.max()
    distanct_matrix_normalized = (distanct_matrix-min_val)/(max_val-min_val)
    proxy_results = {}
    proximity_values = []
    for i in range(len(distanct_matrix_normalized)):
        min_val = np.argmin(distanct_matrix_normalized[i])
        if distanct_matrix_normalized[i][min_val] < proximity_threshold:
            proxy_results[i]=min_val
            proximity_values.append(distanct_matrix_normalized[i][min_val])
        else:
            proxy_results[i]=None
            proximity_values.append(distanct_matrix_normalized[i][min_val])
    return proxy_results,proximity_values

In [84]:
face_proximity_detecton(f1,f2,0.1)

({0: 0, 1: 1}, [0.0, 0.0011550568328700386])

In [38]:
import pickle

In [62]:

class Frame:
    def __init__(self, frame_number):
        self.frame_number = frame_number
        self.frame_array = None
        self.face_info = []

    def insert_face_count(self, face_count):
        self.face_count = face_count

    def insert_frame_array(self, frame_array):
        self.frame_array = frame_array

    def insert_face_info(self, face_boxes, facial_features):
        for box, feature in zip(face_boxes, facial_features):
            self.face_info.append(
                {"face_box": box,
                 "emotion": feature['emotion_output'],
                 "age": feature['age_output'],
                 "sex": feature['gender_output']})

    def get_all_face_positions(self):
        if len(self.face_info) > 0:
            face_boxes = []
            for face in self.face_info:
                x,y,w,h= face['face_box']
                face_boxes.append([x,y])
            return face_boxes
        else:
            return []


class Video:
    def __init__(self, video_name, video_path, total_frames, fps):
        self.name = video_name
        self.path = video_path
        self.total_frames = total_frames
        self.fps = fps
        self.batch_size = int(total_frames / fps)
        self.frame_computer = []
        self.all_frames = []

    def put_frame(self, frame: Frame):
        self.frame_computer.append(frame)
        if len(self.frame_computer) >= self.batch_size:
            yield self.frame_computer
            self.all_frames.extend(self.frame_computer)
            self.frame_computer = []
        else:
            return None

    def get_all_frames(self):
        return self.all_frames


In [90]:
with open('/tmp/video_obj.pkl','rb') as fp:
    video = pickle.load(fp)

In [87]:
video.get_all_frames()[0].get_all_face_positions()

[[303, 460], [100, 654], [481, 770], [194, 319], [524, 439]]

In [88]:
f1 = video.get_all_frames()[4].get_all_face_positions()
f2 = video.get_all_frames()[5].get_all_face_positions()

IndexError: list index out of range

In [78]:
print(f1)
print(f2)

[[147, 123], [556, 207]]
[[146, 122], [556, 209], [653, 174]]


In [106]:
all_frames = video.get_all_frames()
for i in range(len(all_frames)-1):
    print(all_frames[i].frame_number)
    print(all_frames[i].get_all_face_positions(),'--->',all_frames[i+1].get_all_face_positions())
    print(face_proximity_detecton(all_frames[i].get_all_face_positions(),all_frames[i+1].get_all_face_positions(),0.1))
    print("\n")
    print("######################################################################")

5
[[303, 460], [100, 654], [481, 770], [194, 319], [524, 439]] ---> [[304, 460], [102, 655], [481, 771], [196, 321], [523, 441]]
({0: 0, 1: 1, 2: 2, 3: 3, 4: 4}, [0.0, 0.002312921264906562, 0.0, 0.0034213393236765944, 0.002312921264906562])


######################################################################
10
[[304, 460], [102, 655], [481, 771], [196, 321], [523, 441]] ---> [[304, 461], [102, 661], [480, 770], [196, 314], [521, 441]]
({0: 0, 1: 1, 2: 2, 3: 3, 4: 4}, [0.0, 0.009300851571950592, 0.0007705077725442112, 0.01116102188634071, 0.0018601703143901183])


######################################################################
15
[[304, 461], [102, 661], [480, 770], [196, 314], [521, 441]] ---> [[304, 465], [104, 658], [481, 770], [196, 312], [524, 447]]
({0: 0, 1: 1, 2: 2, 3: 3, 4: 4}, [0.005577179521656118, 0.004843875738714247, 0.0, 0.0018590598405520394, 0.010611892692590802])


######################################################################
20
[[304, 465], [104, 

In [110]:
f1 = None
f2 = None
for i in range(len(all_frames)):
    if all_frames[i].frame_number == 35:
        f1=all_frames[i]
        f2 = all_frames[i+1]

In [114]:
f1.face_info

[{'face_box': [305, 467, 187, 249],
  'emotion': {'value': 'sadness', 'probability': '0.91'},
  'age': {'value': 'young', 'probability': '1.00'},
  'sex': {'value': 'male', 'probability': '1.00'}},
 {'face_box': [105, 664, 193, 260],
  'emotion': {'value': 'happiness', 'probability': '0.51'},
  'age': {'value': 'young', 'probability': '1.00'},
  'sex': {'value': 'female', 'probability': '1.00'}},
 {'face_box': [479, 774, 168, 226],
  'emotion': {'value': 'happiness', 'probability': '1.00'},
  'age': {'value': 'middle', 'probability': '0.99'},
  'sex': {'value': 'female', 'probability': '0.94'}},
 {'face_box': [199, 318, 143, 199],
  'emotion': {'value': 'neutral', 'probability': '1.00'},
  'age': {'value': 'young', 'probability': '1.00'},
  'sex': {'value': 'male', 'probability': '1.00'}},
 {'face_box': [525, 446, 173, 223],
  'emotion': {'value': 'anger', 'probability': '0.75'},
  'age': {'value': 'young', 'probability': '0.93'},
  'sex': {'value': 'male', 'probability': '1.00'}}]